# 05-3 트리의 앙상블
##### 앙상블 학습이 무엇인지 이해하고 다양한 앙상블 학습 알고리즘을 실습을 통해 배웁니다.

- 베스트 머신러닝 알고리즘을 찾아보자

### 정형 데이터와 비정형 데이터
- 정형 데이터 : csv, db, 엑셀에 저장하기 쉬운 데이터
- 비정형 데이터 : 책의 글, 이미지, 음악 데이터 등
- 정형 데이터를 다루는데 가장 뛰어난 성과를 내는 알고리즘이 앙상블 학습(ensemble learning)
- 비정형 데이터에는 어떤 알고리즘을 사용할까?
  - 7장에서 배울 신경망 알고리즘
- 사이킷런에서 제공하는 정형 데이터의 끝판왕인 앙상블 학습 알고리즘을 알아보자

### 랜덤 포레스트
- 랜덤 포레스트(random forest)
  - 앙상블 학습의 대표 주자 중 하나
  - 안정적인 성능 덕분에 널리 사용되고 있음
  - 앙상블 학습시 가장 먼저 랜덤 포레스트를 시도해보면 좋을 것 같음
- 각 트리 훈련을 위해 데이터를 랜덤하게 만듦
  - 부트스트랩 샘플(bootstrap sample)
    - 훈련 데이터에서 랜덤하게 샘플을 추출하여 훈련 데이터를 만듦 (한 샘플이 중복 추출 가능)
    - 가방에서 100개 샘플을 뽑는다면 1개를 뽑고 다시 가방에 넣은 후 또 뽑음
    - 기본적으로 부트스트랩 샘플은 훈련 세트의 크기와 같게 만듦
    - 1000개 샘플이 있는 가방에서 중복하여 1000개 샘플을 뽑음
  - 각 노드 분할 시 전체 특성 중 일부 특성을 무작위로 고른 다음 이 중서 최선의 분할을 찾음
    - RandomForestClassifier는 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택
    - 즉 4개의 특성이 있다면 노드마다 2개를 랜덤하게 선택하여 사용
    - (회귀 모델인 RandomForestRegressor는 전체 특성을 사용)
- 사이킷런의 랜덤 포레스트는 기본적으로 100개의 결정 트리를 이런 방식으로 훈련
- 그 다음 분류일 때는 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 정의
- 회귀일때는 단순히 각 트리의 예측을 평균
- 랜덤 포레스트는 랜덤하게 선택한 샘플과 특성을 사용
  - 훈련 세트에 과대적합되는 것을 막아줌
  - 검증 세트와 테스트 세트에서 안정적인 성능 얻을 수 있음
- RandomForestClassifier 클래스로 화이트 와인을 분류하는 문제에 적용

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

- cross_validate() 함수를 사용해 교차 검증 수행
- RandomForestClassifier는 기본적으로 100개의 결정 트리를 사용하므로 n_jobs 매개변수를 -1로 지정하고 모든 CPU 코어를 사용하는 것이 좋음
- cross_validate() 함수의 n_jobs 매개변수도 -1로 지정하여 최대한 병렬로 교차 검증 수행
- return_train_score 매개변수를 True로 지정하면 검증 점수뿐 아니라 훈련 세트에 대한 점수도 같이 반환
- 훈련 세트와 검증 세트의 점수를 비교하면 과대적합을 파악하는데 용이(return_train_score 매개변수의 기본값은 False)

In [5]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)

scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


- 출력 결과는 훈련 세트에 다소 과대적합된 듯함.
- 알고리즘을 조사하는 것이 목적이므로 매개변수를 더 조정하지 않음 (해당 예제는 매우 간단하고 특성이 많지 않아 그리드 서치를 사용해도 튜닝 결과가 크게 나아지지 않음)
- 랜덤 포레스트는 결정 트리의 앙상블이기 때문에 DecisionTreeClassifier가 제공하는 중요한 매개변수를 모두 제공
  - criterion, max_depth, max_features, min_samples_split, min_impurity_decrease, min_samples_leaf 등
- 결정 트리의 가장 큰 장점 중 하나인 특성 중요도를 계산함
- 랜덤 포레스트 특성 중요도는 각 결정 트리의 특성 중요도를 취합한 것
- 랜덤 포레스트 모델 훈련 후 특성 중요도를 출력해보자

In [6]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


- 1절의 결정트리의 특성 중요도는 아래와 같았음
```python
[0.12345626 0.86862934 0.0079144]
```

- 랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련함 -> 하나의 특성에 집중하지 않고 많은 특성이 훈련할 기회를 가짐
  - 과대적합을 줄이고 일반화 성능을 높임
- RandomForestClassifier에는 자체적으로 모델을 평가하는 점수를 얻을 수 있음
  - 랜덤 포레스트는 훈련세트 중복을 허용하여 훈련 -> 훈련하지 않은 남는 샘플(OOB, out of bag)이 생김
  - 이 남는 샘플을 사용하여 훈련 결과를 평가 -> 검증 세트의 역할처럼!
  - 이를 위해서 RandomForestClassifier 클래스 oob_score 매개변수를 True로 지정해야 함

In [7]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


- 교차 검증에서 얻은 점수와 비슷한 결과를 얻음
- OOB 점수는 교차 검증을 대신할 수 있음 -> 훈련 세트에 더 많은 샘플 사용 가능

### 엑스트라 트리